<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/18eV9IhfH8KU3yj332an4GhpxqcT2gHnF?usp=sharing)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

##**Guidance: Structured_LLM_Generation**
Guidance is an efficient programming paradigm for steering language models. With Guidance, you can control how output is structured and get high-quality output for your use case—while reducing latency and cost vs. conventional prompting or fine-tuning. It allows users to constrain generation (e.g. with regex and CFGs) as well as to interleave control (conditionals, loops, tool use) and generation seamlessly.

###**Setup**

In [ ]:
!pip install guidance gpustat llama-cpp-python

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from guidance import models

gpt4 = models.OpenAI("gpt-4o")

###**Simple generation**

In [ ]:
lm = gpt4 + "Who won the last Kentucky derby and by how much?"

###**Selecting among alternatives**

In [ ]:
from guidance import select

gpt4 + f'''\
Q: {query}
Now I will choose to either SEARCH the web or RESPOND.
Choice: {select(["SEARCH", "RESPOND"], name="choice")}'''

###**Interleaved generation and control**

In [ ]:
@guidance
def qa_bot(lm, query):
    lm += f'''\
    Q: {query}
    Now I will choose to either SEARCH the web or RESPOND.
    Choice: {select(["SEARCH", "RESPOND"], name="choice")}
    '''
    if lm["choice"] == "SEARCH":
        lm += "A: I don't know, Google it!"
    else:
        lm += f'A: {gen(stop="Q:", name="answer")}'
    return lm

gpt4 + qa_bot(query)

###**CHAT**

In [ ]:
gpt4o = models.OpenAI("gpt-4o")

In [ ]:
from guidance import system, user, assistant

with system():
    lm = gpt4o + "You are a helpful assistant."

with user():
    lm += "What is the meaning of life?"

with assistant():
    lm += gen("response")

###**Multistep**

In [ ]:
@guidance
def experts(lm, query):
    with system():
        lm += "You are a helpful assistant."

    with user():
        lm += f"""\
        I want a response to the following question:
        {query}
        Who are 3 world-class experts (past or present) who would be great at answering this?
        Please don't answer the question or comment on it yet."""

    with assistant():
        lm += gen(name='experts', max_tokens=300)

    with user():
        lm += f"""\
        Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
        In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
        If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
        Please start your answer with ANSWER:"""

    with assistant():
        lm += gen(name='answer', max_tokens=500)

    return lm

gpt4 + experts(query='What is the meaning of life?')

###**Invalid output without a regex guide**

In [ ]:
from guidance import models, gen

gpt4o = models.OpenAI("gpt-4o")

gpt4o + f"""\
Tweak this proverb to apply to model instructions instead.

Where there is no guidance, a people falls,
but in an abundance of counselors there is safety.
- Proverbs 11:14

UPDATED
Where there is no guidance{gen('rewrite', stop="- ")}
- GPT {gen('chapter', max_tokens=10)}:{gen('verse', max_tokens=10)}"""

###**Valid output with a regex guide**

In [ ]:
gpt4o + f"""\
Tweak this proverb to apply to model instructions instead.

Where there is no guidance, a people falls,
but in an abundance of counselors there is safety.
- Proverbs 11:14

UPDATED
Where there is no guidance{gen('rewrite', stop="- ")}
- GPT {gen('chapter', regex="[0-9]+")}:{gen('verse', regex="[0-9]+")}"""

###**With token healing**

In [ ]:
import guidance
from guidance import gen, models


gpt4o += "The url of Google is http:" + gen(max_tokens=5)